In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2022-12-27 18:05:30.350696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 18:05:30.673265: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-27 18:05:30.673286: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-27 18:05:32.018679: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
path_to_file = "../Project/WikiCreole_Texts/Toscane.txt"

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 50479 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

{{Voir homonymes|Toscane (homonymie)}}
{{coord|43.5|11.2|format=dec|region:IT-52_type:landmark_scale:400000|display=title}}
{{Infobox Région d'Italie
 | nom             = Toscane
 | blason          = Coat of arms of Tuscany.svg
 | drapeau         = F


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

117 unique characters


In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-12-27 18:05:33.803168: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-27 18:05:33.803390: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-27 18:05:33.803411: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (diego-Inspiron-5370): /proc/driver/nvidia/version does not exist
2022-12-27 18:05:33.803659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[62, 63, 64, 65, 66, 67, 68], [85, 86, 87]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
# Back to string

tf.strings.reduce_join(chars, axis=-1).numpy()

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [11]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(50479,), dtype=int64, numpy=array([88, 88, 54, ..., 60,  1,  1])>

In [12]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [13]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

{
{
V
o
i
r
 
h
o
m


In [14]:
seq_length = 100

In [15]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq)) 

tf.Tensor(
[b'{' b'{' b'V' b'o' b'i' b'r' b' ' b'h' b'o' b'm' b'o' b'n' b'y' b'm'
 b'e' b's' b'|' b'T' b'o' b's' b'c' b'a' b'n' b'e' b' ' b'(' b'h' b'o'
 b'm' b'o' b'n' b'y' b'm' b'i' b'e' b')' b'}' b'}' b'\n' b'{' b'{' b'c'
 b'o' b'o' b'r' b'd' b'|' b'4' b'3' b'.' b'5' b'|' b'1' b'1' b'.' b'2'
 b'|' b'f' b'o' b'r' b'm' b'a' b't' b'=' b'd' b'e' b'c' b'|' b'r' b'e'
 b'g' b'i' b'o' b'n' b':' b'I' b'T' b'-' b'5' b'2' b'_' b't' b'y' b'p'
 b'e' b':' b'l' b'a' b'n' b'd' b'm' b'a' b'r' b'k' b'_' b's' b'c' b'a'
 b'l' b'e' b':'], shape=(101,), dtype=string)


In [16]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'{{Voir homonymes|Toscane (homonymie)}}\n{{coord|43.5|11.2|format=dec|region:IT-52_type:landmark_scale:'
b"400000|display=title}}\n{{Infobox R\xc3\xa9gion d'Italie\n | nom             = Toscane\n | blason          = Co"
b'at of arms of Tuscany.svg\n | drapeau         = Flag of Tuscany.svg\n | imageloc        = Tuscany in It'
b'aly.svg\n | zone            = ITE\n | capitale        = [[Florence]]\n | gentil\xc3\xa9         = Toscans / Tos'
b'canes\n | latitude        = 43.41\n | longitude       = 11\n | pr\xc3\xa9sident       = [[Eugenio Giani]]\n | pa'


In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [18]:
# Exemple

split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [19]:
dataset = sequences.map(split_input_target)

In [20]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'{{Voir homonymes|Toscane (homonymie)}}\n{{coord|43.5|11.2|format=dec|region:IT-52_type:landmark_scale'
Target: b'{Voir homonymes|Toscane (homonymie)}}\n{{coord|43.5|11.2|format=dec|region:IT-52_type:landmark_scale:'


In [21]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [22]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [23]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [24]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 118) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  30208     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  120950    
                                                                 
Total params: 4,089,462
Trainable params: 4,089,462
Non-trainable params: 0
_________________________________________________________________


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [28]:
sampled_indices

array([ 21,  55, 113,   7,  42,  93,   8,   3,  40, 106,  98,  58, 114,
        50,  76,  37, 116,  71,  77,  47,   3,  57, 114,  86,   3, 100,
        46,   0,   6,  22,  29,  98,  93, 112,  41,  26,   2,  82,  59,
       100, 117,  17,  32,   5,  59,  32,  52,  36,  55,  31,  54,  34,
        99, 100,  38,  99,  90,  25,  40,  71,   5,   6,  77,  34, 113,
        55,  97,  46,  22,  30,  32, 117, 117,  99, 109, 104,   0,  22,
        32,  37,   7,  20,   1, 107, 109,  79,  90,  99,  77,  12, 100,
         2, 117, 113,  50,  37,  84, 106,  33, 106])

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'p://www.persee.fr/doc/befar_0257-4101_1983_mon_249_1?h=celtique&h=po |consult\xc3\xa9 le=31 janvier 2016 |p'

Next Char Predictions:
 b"4W\xc5\x99&J\xc3\x80'!H\xc3\xaf\xc3\xa1Z\xcb\x88RoE\xe2\x80\x94jpO!Y\xcb\x88y!\xc3\xa7N[UNK]%5<\xc3\xa1\xc3\x80\xc5\x93I9 u[\xc3\xa7\xe2\x80\x990?#[?TDW>VB\xc3\xa2\xc3\xa7F\xc3\xa2}8Hj#%pB\xc5\x99W\xc3\xa0N5=?\xe2\x80\x99\xe2\x80\x99\xc3\xa2\xc3\xbb\xc3\xad[UNK]5?E&3\n\xc3\xb4\xc3\xbbr}\xc3\xa2p+\xc3\xa7 \xe2\x80\x99\xc5\x99REw\xc3\xafA\xc3\xaf"


In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 118)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.7695274, shape=(), dtype=float32)


In [32]:
tf.exp(example_batch_mean_loss).numpy()

117.86353

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
EPOCHS = 100

In [36]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
7/7 [==============================] - 17s 2s/step - loss: 4.8229
Epoch 2/100
7/7 [==============================] - 15s 2s/step - loss: 4.2448
Epoch 3/100
7/7 [==============================] - 16s 2s/step - loss: 3.9728
Epoch 4/100
7/7 [==============================] - 16s 2s/step - loss: 3.7038
Epoch 5/100
7/7 [==============================] - 15s 2s/step - loss: 3.5392
Epoch 6/100
7/7 [==============================] - 15s 2s/step - loss: 3.4101
Epoch 7/100
7/7 [==============================] - 15s 2s/step - loss: 3.2945
Epoch 8/100
7/7 [==============================] - 15s 2s/step - loss: 3.1271
Epoch 9/100
7/7 [==============================] - 16s 2s/step - loss: 2.9813
Epoch 10/100
7/7 [==============================] - 16s 2s/step - loss: 2.8624
Epoch 11/100
7/7 [==============================] - 16s 2s/step - loss: 2.7837
Epoch 12/100
7/7 [==============================] - 17s 2s/step - loss: 2.7081
Epoch 13/100
7/7 [==============================] - 16s 2s/st

In [37]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [38]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [39]:
start = time.time()
states = None
next_char = tf.constant(['{'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


{{formatnum:16061}}
|0,90
|—
|bgcolor="#FC2D2D"|
|align=left| [[Parti communiste italien (2016)|Parti communi titite (Italie)|CD]])
|{{formatnum:47647}}
|2,95
|align=center| {{augmentation||8px}} 3
|-
|{{Infobox Parti politique italien/couleurs|PD}}|
|34,71 ||'''22''' 
|-
|{{Infobox Parti politique italien/couleurs|FI}}|
|align=left|  [[Forza Italia (2013)|Forza Italia]]-[[Utalie'e comine|Vité]] fur couvrer communché tesc avec [[Itrlen]].
Ens [[Cherre de Chanco]], [[Charles VIII d'Étaut et drépor| {{augmentation||8px}} 3
| rowspan="4" bgcolor="#eeeeee" |
|-
|{{Infobox Parti politique italien/couleurs|Lega}}|
|align=left|  [[Ligue du Nord|Ligue]] (Lega)
|{{formatnum:351977}}
|21,78
|align=center| {{augmentation||8px}} 11,7
| '''12'''
|align=center| {{augmentation||8px}} 3
|-
|{{Infobox Parti politique italien/couleurs|PD}}|
|align=left| [[Parti démocrate (Italie)|PD]]
|bgcolor="#FFB2B2"|33,43 ||bgcolor="#FFB2B2"|2 
|bgcolor="#FFB2B2"|37,33 ||bgcolor="#FFB2B2"|1 
|bgcolor="#FFB2B2"|37,35

In [40]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [41]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:128100]] ;
* [[Province d'Arezzo|Arezzo]] ;
* [[Province de Sienne|Sienne]].
}}

== Culture ==

La r
